In [2]:
import pandas as pd
import os

from datetime import datetime, date, timedelta
from time import time
from dateutil.relativedelta import *

import shutil

import numpy as np
import math
from collections import Counter

import calendar
from pathlib import Path

from utils import setup_logging 

import json

import requests
import urllib.parse

import sys


In [308]:
def download_stock_row_statements(ticker,data_path):
    stockrow_folder = f'{data_path}stockrow/{ticker}/'
    Path(stockrow_folder).mkdir(parents=True, exist_ok=True)



    for statement in ['Income Statement','Cash Flow','Balance Sheet']:

        print(f'{stockrow_folder}/{statement}.xlsx')
        if os.path.exists(f'{stockrow_folder}/{statement}.xlsx'): 
            print(f"Didn't download {statement} for {ticker} because file already exists")
        else: 
            url = f'https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=Q&section={urllib.parse.quote(statement)}&sort=desc'
            try:
                r = requests.get(url, allow_redirects=True)
            except: 
                print(f'Failed to download {statement} for {ticker}')


            if (b'The page you were looking for doesn\'t exist (404)' in r.content):
                stockrow_logger.warning(f"{statement}.xlsx' not found") 
            else:
                open(f'{stockrow_folder}/{statement}.xlsx', 'wb').write(r.content)


def lower_index(df):
    df.index = df.index.str.lower()
    return df      

def load_statement(data_path,ticker,statement):
    df = pd.read_csv(f"{data_path}timeseries/{ticker}/Canonical Statement/{statement}.csv",index_col=[2])  
    df = df.drop(['filing_label','xbrl_tag'],axis=1)
    df.columns = list_of_rounded_months(df.columns)
    df.columns = pd.to_datetime(df.columns)

    df = lower_index(df)

    df_sr = pd.read_excel(f"{data_path}stockrow/{ticker}/{statement}.xlsx",index_col=[0])  
    df_sr = lower_index(df_sr)

    return df, df_sr

def match_rows_and_columns(df,df_sr):
    matched_labels = df[df.index.isin(df_sr.index)].index
    matched_dates = df.loc[:,df.columns.isin(df_sr.columns.astype(str))].columns
    
    df = df.loc[matched_labels,matched_dates]
    df_sr = df_sr.loc[matched_labels,matched_dates]
    df_sr.index = [f'{index}_stockrow' for index in df_sr.index]

    return df,df_sr

def missing_rows(data_path,df,df_sr):
    with open(f"{data_path}/mappings/canonical_label_tag_mapping.json") as json_file:
        mapping_dict = json.load(json_file)
        
    canonical_keys = [key.lower() for key in mapping_dict[statement].keys()]

    missing_rows = pd.Index(canonical_keys)[~pd.Index(canonical_keys).isin(df.index)]
    missing_rows_in_stock_row = df_sr[df_sr.index.isin(missing_rows)].index
    
    return missing_rows, missing_rows_in_stock_row

def missing_columns(df,df_sr):
    missing_dates = df_sr.columns[~df_sr.columns.isin(df.columns)]
    return [datetime.strftime(date,'%Y-%m-%d') for date in missing_dates]
    
def find_discrepancy(df,df_sr,label):

    df_matched, df_sr_matched = match_rows_and_columns(df,df_sr)
    not_matched = df_matched.loc[label,:] != df_sr_matched.loc[f'{label}_stockrow',:]

    return pd.DataFrame(df_matched.loc[label,not_matched]).merge(df_sr_matched.loc[f'{label}_stockrow',not_matched],left_index=True,right_index=True)

def label_discrepancies(df,df_sr):
    label_dict = {}
    for label in df.index:
        if (label is not np.nan) and (label in df.index) and (label in df_sr.index):
            label_dict[label] = find_discrepancy(df,df_sr,label)
    return label_dict

#for label in (df_matched == df_sr_matched).all(axis=1):

def load_stockrow_statement(ticker,data_path,statement_type,stockrow_logger):
    statement_folder = f'{data_path}stockrow/{ticker}/'
    df_comparison = pd.read_excel(f'{statement_folder}{statement_type}.xlsx',index_col=[0])
    df_comparison.columns = pd.DatetimeIndex(df_comparison.columns)
    return df_comparison
                
def list_of_rounded_months(date_list):
    rounded_date_list = []

    for date_str in date_list:

        rounded_date = date_round(date_str)
        rounded_date_list.append(rounded_date)

    return rounded_date_list


def date_round(date_str):
    year = int(date_str[0:4])

    month = int(date_str[5:7])
    day = int(date_str[8:10])

    if round(day/calendar.monthrange(year,month)[1],0) == 1:
        rounded_date = (date(year, month, 1) + relativedelta(months=+1) - timedelta(days=1))
    else:
        rounded_date = (date(year, month, 1) - timedelta(days=1))

    return rounded_date

def run_stockrow_check(data_path,ticker):
    for statement in ['Income Statement','Balance Sheet', 'Cash Flow']:
        download_stock_row_statements(ticker,data_path)

        df, df_sr = load_statement(data_path,ticker,statement)
        missing_labels, missing_labels_in_stock_row = missing_rows(data_path,df,df_sr)
        missing_dates = missing_columns(df,df_sr)
        label_discrepancy_dict = label_discrepancies(df,df_sr)

        print(f"missing labels: {', '.join(missing_labels)}")
        print(f"missing labels in stockrow: {', '.join(missing_labels_in_stock_row)}\n")
        print(f"missing dates: {', '.join(missing_dates)}\n")
        for label in label_discrepancy_dict:
            print(label_discrepancy_dict[label].T)
            print("\n")





In [309]:
data_path = '../data/'
ticker = 'AAPL'
run_stockrow_check(data_path,ticker)
    


../data/stockrow/AAPL//Income Statement.xlsx
Didn't download Income Statement for AAPL because file already exists
../data/stockrow/AAPL//Cash Flow.xlsx
Didn't download Cash Flow for AAPL because file already exists
../data/stockrow/AAPL//Balance Sheet.xlsx
Didn't download Balance Sheet for AAPL because file already exists
missing labels: interest expenses, net income common, net income from non-controlling interests, income from discontinued operations, income from continuous operations, share of subsidiary income, eps (basic, from continuous ops), eps (diluted, from continous ops), dividends (preferred)
missing labels in stockrow: net income common, income from continuous operations, eps (basic, from continuous ops)

missing dates: 

                    2017-09-30    2012-09-30
revenue                    NaN           NaN
revenue_stockrow  5.257900e+10  3.596600e+10


                            2017-09-30    2012-09-30
cost of revenue                    NaN           NaN
cost of rev

,revenue,revenue_stockrow
2019-12-31,-9.031000e+09,5.204000e+09
2019-03-31,1.964900e+10,5.414000e+09
2018-12-31,3.848700e+10,-1.045600e+10
2018-09-30,2.012300e+10,5.683000e+09
2017-12-31,NaN,-3.074600e+10
2017-06-30,NaN,1.383400e+10
2017-03-31,NaN,1.323000e+10
2016-09-30,NaN,1.248300e+10


In [188]:
#print missing rows
#print missing dates
#print discrepancy in values
df_sr[~df.index.isin(df_sr.index)].index

Index([                                        nan,
                                               nan,
                                               nan,
                                               nan,
                                               nan,
                      'share of subsidiary income',
                            'non-operating income',
                               'interest expenses',
                               'income before tax',
                                               nan,
                                      'net income',
       'net income from non-controlling interests',
                                               nan,
                                               nan,
               'eps (diluted, from continous ops)',
                                               nan,
                'shares (basic, weighted average)',
              'shares (diluted, weighted average)',
                                               nan,
            

In [176]:
s_dict = {}
sr_dict = {}
for statement in ['Income Statement','Cash Flow','Balance Sheet']:
    #s_dict[statement] = pd.read_csv(f"{data_path}timeseries/{ticker}/Canonical Statement/{statement}.csv",index_col=[0,1,2])  
    s_dict[statement] = pd.read_csv(f"{data_path}timeseries/{ticker}/Canonical Statement/{statement}.csv",index_col=[2])  
    s_dict[statement] = s_dict[statement].drop(['filing_label','xbrl_tag'],axis=1)
    s_dict[statement].columns = pd.to_datetime(s_dict[statement].columns)
    s_dict[statement] = lower_index(s_dict[statement])
    
    sr_dict[statement] = pd.read_excel(f"{data_path}stockrow/{ticker}/{statement}.xlsx",index_col=[0])  
    sr_dict[statement] = lower_index(sr_dict[statement])

statement = 'Income Statement'
matched_labels = s_dict[statement][s_dict[statement].index.isin(sr_dict[statement].index)].index
matched_dates = s_dict[statement].loc[:,s_dict[statement].columns.isin(sr_dict[statement].columns.astype(str))].columns

df_sr = sr_dict[statement].loc[matched_labels,matched_dates]
df = s_dict[statement].loc[matched_labels,matched_dates] 
df_zcompare = df == df_sr

df_sr.index = [f'{index}_stockrow' for index in df_sr.index]
df_zcompare.index = [f'{index}_zcompare' for index in df_zcompare.index]

df = df.append(df_sr)
df = df.append(df_zcompare)
df.sort_index()

,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-09-30
cost of revenue,3.318000e+09,-7.697000e+09,3.531000e+09,3.496000e+09,1.472600e+10,1.556700e+10,1.547700e+10,1.797400e+10,1.631500e+10,1.728900e+10,1.217000e+10,1.076100e+10,1.019400e+10,9.840000e+09
cost of revenue_stockrow,3.318000e+09,3.408000e+09,3.531000e+09,3.496000e+09,3.621000e+09,-8.868000e+09,3.770000e+09,4.085000e+09,1.631500e+10,-2.358300e+10,1.218600e+10,1.076100e+10,1.019400e+10,9.840000e+09
cost of revenue_zcompare,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
eps (basic),-8.300000e-01,7.000000e-01,5.000000e-01,-7.600000e-01,2.300000e-01,2.100000e-01,2.200000e-01,7.600000e-01,4.700000e-01,-1.220000e+00,3.200000e-01,1.080000e+00,7.400000e-01,6.400000e-01
eps (basic)_stockrow,-8.300000e-01,2.400000e-01,5.000000e-01,-7.600000e-01,6.901000e-01,6.301000e-01,6.601000e-01,2.280200e+00,1.410100e+00,-1.620200e+00,9.601000e-01,3.240300e+00,2.220200e+00,1.920200e+00
eps (basic)_zcompare,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"eps (basic, from continuous ops)",-8.300000e-01,-1.000000e-01,4.900000e-01,-1.480000e+00,2.300000e-01,2.100000e-01,2.200000e-01,7.600000e-01,4.700000e-01,-1.180000e+00,3.300000e-01,1.080000e+00,7.400000e-01,6.400000e-01
"eps (basic, from continuous ops)_stockrow",-8.300000e-01,2.400000e-01,4.900000e-01,-1.480000e+00,-1.100000e-01,-1.130100e+00,1.500000e-01,3.000000e-02,1.410100e+00,-6.060600e+00,9.901000e-01,3.240300e+00,2.220200e+00,1.920200e+00
"eps (basic, from continuous ops)_zcompare",1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
eps (diluted),-8.300000e-01,7.000000e-01,5.000000e-01,-7.600000e-01,2.300000e-01,2.100000e-01,2.100000e-01,7.600000e-01,4.700000e-01,-1.200000e+00,3.200000e-01,1.070000e+00,7.200000e-01,6.300000e-01


In [164]:
s_dict[statement].loc[matched_labels,matched_dates] == sr_dict[statement].loc[matched_labels,matched_dates]

,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-09-30
standard_label,,,,,,,,,,,,,,
revenue,True,False,True,True,False,False,False,True,True,False,True,False,False,False
cost of revenue,True,False,True,True,False,False,False,False,True,False,False,True,True,True
r&d expenses,True,False,True,True,False,False,False,False,True,False,False,True,True,True
sg&a expenses,True,False,True,True,False,False,False,False,True,False,False,True,True,True
income tax provision,True,False,True,True,False,False,False,False,True,False,True,True,True,True
income from continuous operations,True,False,True,True,False,False,False,False,True,False,True,True,True,True
"eps (basic, from continuous ops)",True,False,True,True,False,False,False,False,False,False,False,False,False,False
eps (basic),True,False,True,True,False,False,False,False,False,False,False,False,False,False
eps (diluted),True,False,True,True,False,False,False,False,False,False,False,False,False,False


<bound method DataFrame.sort_index of                                                       2020-03-31  \
revenue                                             5.221000e+09   
cost of revenue                                     3.318000e+09   
r&d expenses                                        2.360000e+08   
sg&a expenses                                       6.330000e+08   
income tax provision                                4.400000e+07   
income from continuous operations                  -6.100000e+08   
eps (basic, from continuous ops)                   -8.300000e-01   
eps (basic)                                        -8.300000e-01   
eps (diluted)                                      -8.300000e-01   
revenue_stockrow                                    5.221000e+09   
cost of revenue_stockrow                            3.318000e+09   
r&d expenses_stockrow                               2.360000e+08   
sg&a expenses_stockrow                              6.330000e+08   
income tax

AttributeError: 'DataFrame' object has no attribute 'sort'

In [160]:
df.shape

(9, 14)

In [161]:
sr_dict[statement].index

Index(['revenue', 'revenue growth', 'cost of revenue', 'gross profit',
       'r&d expenses', 'sg&a expenses', 'operating income',
       'non-operating interest expenses', 'non-operating income/expense',
       'ebt', 'income tax provision', 'income after tax',
       'dividends (preferred)', 'non-controlling interest',
       'net income common', 'eps (basic)', 'eps (diluted)',
       'shares (basic, weighted)', 'shares (diluted, weighted)',
       'gross margin', 'ebit margin', 'ebt margin', 'net profit margin',
       'ebitda', 'ebit', 'income from continuous operations',
       'income from discontinued operations', 'consolidated net income/loss',
       'eps (basic, from continuous ops)', 'eps (diluted, from cont. ops)',
       'eps (diluted, from disc. ops)', 'eps (basic, from discontinued ops)',
       'eps (basic, consolidated)', 'eps (diluted, consolidated)',
       'shares (diluted, average)', 'ebitda margin',
       'operating cash flow margin'],
      dtype='object')

In [58]:
#s_dict[statement][matched_label_bool] 
sr_dict[statement].loc[matched_labels,:] 

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [20]:
sr_dict[statement]

,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,...,2012-12-31,2012-09-30,2012-06-30,2012-03-31,2011-12-31,2011-09-30,2011-06-30,2011-03-31,2010-12-31,2010-09-30
Revenue,5.221000e+09,5.204000e+09,5.426000e+09,5.468000e+09,5.414000e+09,-1.045600e+10,5.683000e+09,5.857000e+09,2.151000e+10,-3.074600e+10,...,1.391700e+10,1.363700e+10,1.451300e+10,1.471900e+10,1.409700e+10,1.510900e+10,1.604600e+10,1.473300e+10,1.377100e+10,1.286800e+10
Revenue Growth,-3.560000e-02,1.497700e+00,-4.520000e-02,-6.640000e-02,-7.483000e-01,6.599000e-01,-6.299000e-01,-5.766000e-01,6.259000e-01,-3.361400e+00,...,-1.280000e-02,-9.740000e-02,-9.550000e-02,-1.000000e-03,2.370000e-02,1.742000e-01,1.783000e-01,9.810000e-02,1.047000e-01,6.820000e-02
Cost of Revenue,3.318000e+09,3.408000e+09,3.531000e+09,3.496000e+09,3.621000e+09,-8.868000e+09,3.770000e+09,4.085000e+09,1.631500e+10,-2.358300e+10,...,1.193900e+10,1.136800e+10,1.220000e+10,1.228500e+10,1.243300e+10,1.292800e+10,1.355100e+10,1.211700e+10,1.181800e+10,1.084100e+10
Gross Profit,1.903000e+09,1.796000e+09,1.895000e+09,1.972000e+09,1.793000e+09,-1.588000e+09,1.913000e+09,1.772000e+09,5.195000e+09,-7.163000e+09,...,1.978000e+09,2.269000e+09,2.313000e+09,2.434000e+09,1.664000e+09,2.181000e+09,2.495000e+09,2.616000e+09,1.953000e+09,2.027000e+09
R&D Expenses,2.360000e+08,2.310000e+08,2.250000e+08,2.320000e+08,2.670000e+08,-2.320000e+08,2.640000e+08,2.700000e+08,7.680000e+08,-6.980000e+08,...,4.630000e+08,4.340000e+08,4.060000e+08,4.050000e+08,4.330000e+08,4.020000e+08,4.110000e+08,4.000000e+08,4.430000e+08,4.030000e+08
SG&A Expenses,6.330000e+08,6.500000e+08,6.450000e+08,6.420000e+08,7.260000e+08,-1.850000e+08,7.310000e+08,7.680000e+08,1.714000e+09,-8.650000e+08,...,7.410000e+08,7.390000e+08,6.740000e+08,7.070000e+08,7.020000e+08,6.910000e+08,6.950000e+08,7.000000e+08,6.590000e+08,6.400000e+08
Operating Income,-4.220000e+08,4.120000e+08,5.840000e+08,-8.320000e+08,-5.400000e+07,-1.626000e+09,1.230000e+08,7.600000e+07,1.635000e+09,-6.285000e+09,...,-5.990000e+08,9.580000e+08,1.138000e+09,8.600000e+08,4.120000e+08,1.010000e+09,1.344000e+09,9.130000e+08,6.880000e+08,8.150000e+08
Non-operating Interest Expenses,1.830000e+08,1.750000e+08,1.770000e+08,1.650000e+08,1.510000e+08,-2.950000e+08,NaN,NaN,3.500000e+08,3.540000e+08,...,3.100000e+08,3.180000e+08,3.120000e+08,3.290000e+08,3.310000e+08,3.050000e+08,3.280000e+08,3.770000e+08,3.680000e+08,3.620000e+08
Non-operating Income/Expense,-1.440000e+08,-2.230000e+08,-1.340000e+08,-1.160000e+08,-1.110000e+08,3.860000e+08,4.500000e+07,5.400000e+07,-9.300000e+07,6.930000e+08,...,-2.510000e+08,-1.330000e+08,-1.540000e+08,-1.540000e+08,-5.800000e+07,7.900000e+07,-2.700000e+07,-7.200000e+07,-4.200000e+07,-1.040000e+08
EBT,-5.660000e+08,1.890000e+08,4.500000e+08,-9.480000e+08,-1.650000e+08,-1.240000e+09,1.680000e+08,1.300000e+08,1.542000e+09,-5.592000e+09,...,-8.500000e+08,8.250000e+08,9.840000e+08,7.060000e+08,3.540000e+08,1.089000e+09,1.317000e+09,8.410000e+08,6.460000e+08,7.110000e+08
